# Exercises:

Answer the following questions.

## 1. Import the `Requests` package and call the [GitHub Jobs](https://jobs.github.com) API with the parameters of your choice. Print the status code to make sure it went well.

In [9]:
import requests
parameters = {'location': 'San Francisco'}
response = requests.get("https://jobs.github.com/positions.json", parameters)
status_code = response.status_code
print(response)
print(status_code)


<Response [200]>
200


## 2. Import `json` and print the whole json response with the `json.dumps()` function. See what kind of data it contains.


In [10]:
import json
data = response.json()
print(json.dumps(data, sort_keys=True, indent="\t"))

[
	{
		"company": "GitHub",
		"company_logo": "http://github-jobs.s3.amazonaws.com/15bbd3fc-d9ab-11e6-996a-6c07a253eabc.png",
		"company_url": "https://jobs.lever.co/github/61b1850e-8a45-4df4-ad09-2f65e1e95cc4",
		"created_at": "Sat Mar 11 16:12:39 UTC 2017",
		"description": "<p>GitHub is looking for an experienced manager to lead a small team within our application engineering group. Together we will create products that are integral to how people build software. We pride ourselves on being thoughtful, intentional and getting things done. Hopefully you do too.</p>\n\n<p>Here are some of the signs of a successful Engineering Manager at GitHub:</p>\n\n<ul>\n<li><p>The team always understands the \u201cwhy\u201d and knows what success looks like for any given product or feature</p></li>\n<li><p>The team builds software that is architecturally consistent and of high quality</p></li>\n<li><p>The team ships products, features and bug fixes regularly</p></li>\n<li><p>Individuals on the team

## 3. Get the dictionary of one of the jobs and call the `keys()` method to have a better idea of the data at hand.

In [11]:
job = data[0]
job.keys()

dict_keys(['id', 'created_at', 'title', 'location', 'type', 'description', 'how_to_apply', 'company', 'company_url', 'company_logo', 'url'])

## 4. Print some of the first job's content, including images and HTML. You might need to import the `IPython` package.

In [12]:
from IPython.display import Image
Image(url = job['company_logo'])
print(job['id'], job['created_at'],job['title'],job['location'],
      job['type'],job['description'],job['how_to_apply'],job['company'],
      job['company_url'],job['url'],sep="\n")
                                                            
                                                                                                                     

1ad19ffc-d9ab-11e6-8c76-6f23f35bb7ca
Sat Mar 11 16:12:39 UTC 2017
Application Engineering Manager
San Francisco
Full Time
<p>GitHub is looking for an experienced manager to lead a small team within our application engineering group. Together we will create products that are integral to how people build software. We pride ourselves on being thoughtful, intentional and getting things done. Hopefully you do too.</p>

<p>Here are some of the signs of a successful Engineering Manager at GitHub:</p>

<ul>
<li><p>The team always understands the “why” and knows what success looks like for any given product or feature</p></li>
<li><p>The team builds software that is architecturally consistent and of high quality</p></li>
<li><p>The team ships products, features and bug fixes regularly</p></li>
<li><p>Individuals on the team are learning, growing and advancing in their respective careers</p></li>
<li><p>The team is responsive to recruiting and interview needs</p></li>
<li><p>Team members are emp

## 5. Import `SQLite` and create a database.

In [13]:
import sqlite3
dbname = 'githubdata.sqlite'
con = sqlite3.connect(dbname)

## 6. Create 2 tables: `Company` and `Job`. Pay special attention on primary and foreign keys as well as unique and non-nullable columns. Make sure you store `created_at` as the equivalent of Python `datetime` and not `string`.


In [14]:
con.execute(
    "CREATE TABLE Company ("
        "id INTEGER NOT NULL,"
        "name TEXT NOT NULL,"
        "url TEXT,"
        "logo_url TEXT,"
        "PRIMARY KEY (id),"
        "UNIQUE (url)"
")")

con.execute(
    "CREATE TABLE Job ("
        "id INTEGER NOT NULL,"
        "company_id INTEGER NOT NULL,"
        "title TEXT NOT NULL,"
        "description TEXT,"
        "created_at TIMESTAMP NOT NULL,"
        "PRIMARY KEY (id),"
        "FOREIGN KEY (company_id) REFERENCES Company (id)"
")")

con.commit()

OperationalError: table Company already exists

## 7. Store the data you got from the API request in the database. Make sure you create a transaction (i.e., if an error occurs while saving a job's location data the database should rollback on all statements regarding that job).

In [15]:
def insert_data(company_name, company_url, company_logo_url, created_at_str, title, description, location_description):
    transaction = sqlite3.connect(dbname)
    with transaction:
        company_id = insert_company(company_name, company_url, company_logo_url, transaction)
        job_id = insert_job(company_id, title, description, location_description, created_at_str, transaction)
    
    return job_id

In [16]:
from datetime import datetime
created_at_pattern = '%a %b %d %X %Z %Y'
from sqlite3 import IntegrityError
def insert_company(name, url, logo_url, connection = con):
    try:
        con.execute("INSERT INTO Company (name, url,logo_url) VALUES (?,?,?)", (name,url,logo_url))
        new_id = con.excute("SELECT last_insert_rowid()").fetchone()
        con.commit()
        return new_id
    except IntegrityError as err:
        print(err)
        print(name)
        
def insert_job(company_id, title, description, location, cretaed_at_str, connection = con):
    try:
        con.execute("INSERT INTO Job (company_id, title, description, location, created_at_str)"
                    "VALUES (?,?,?,?,?)", 
                    (company_id, title, description, location, datetime.strptime(created_at_str, created_at_pattern)))
        new_id = con.excute("SELECT last_insert_rowid()").fetchone()
        con.commit()
        return new_id
    except IntegrityError as err:
        print(err)
        print(name)
        

        
        

## 8. Execute simple SQL queries in each table and make sure everything looks as was intended.

In [20]:
for row in con.execute("SELECT * FROM Company"):
    print('\n {}'.format(tuple(row)))
    

## 9. Create queries the following queries:
 * Find out how many jobs have the phrase `Data Science` and how many have the word `Python` in the description.
 * Find out how many were created in the past 7 days.
 * Find out how many jobs are located in `California` and which companies those are with.

**The output from our queries should be similar to this:**

There are 27 jobs with the 'Data Science' term, and 40% of them have 'Python' in the description.

22% of them have been created in the past 7 days.

2 of those are in California. The companies with those jobs are: ['Carbon Black, Inc.', 'Kaggle']

In [23]:
cursor = con.execute("SELECT COUNT() FROM Job")
total, = cursor.fetchone()
cursor = con.execute("SELECT COUNT() FROM Job WHERE description LIKE '%python%' ")
python, = cursor.fetchone()
print("/n There are {} jobs with datascience terma and {}% of them have python in description"
     .format(total, 100 * python // total))


ZeroDivisionError: integer division or modulo by zero

In [24]:
from datetime import datetime , timedelta
cursor = con.execute("SELECT COUNT() FROM Job WHERE created_at >= :week_ago",
                    {"week_ago": datetime.today() - timedelta(days = 7)})

last_week, = cursor.fetchone()
print("\n {}% of them have been created in the past 7 days"
     .format( 100 * last_week // total))

ZeroDivisionError: integer division or modulo by zero

In [ ]:
cursor = con.execute("SELECT COUNT() FROM Job WHERE location LIKE '%CA%' OR location LIKE '%California%' ")
ca_jobs, = cursor.fetchone()
if ca_jobs > 0:
    cursor =con.execute("SELECT name FROM Company JOIN job ON company.id =job.company_id"
                       "WHERE location LIKE'%CA%' OR location LIKE '%California%' ")
    ca_companies = cursor.fetchall()
    if len(ca_companies) ==1:
        message = '\n{} of those based in California. The company with that job is {}'
    else:
         message = '\n{} of those based in California. The company with that job are: {}'
            
    print(message.format(ca_jobs, "; ".join(str(company) for company, in ca_companies))